# ML-Fundamentals - Neural Networks - Exercise: Neural Network Framework

# Tasks
Your main goal is to extend the existing framework, to perform experiments with different model combinations and to document your observations. Here is a list of necessary tasks and some ideas for additional points:
  * (6) Write a data loader for a different image dataset, e.g., CIFAR or Labelled Faces in the Wild. Feel free to search a dataset you like to classify. Create and train a simple fully connected network on that dataset in this notebook.
  * (10) Implement the `Conv` and `Pool` Layer in `layer.py`. Create and train a convolutional neural network on Mnist and your chosen dataset in this notebook.

Bonus points
  * (5) 1 to 5 points are given for improving the class and method comments in the framework files. Points are given based on the quality and quantity of the comments.
  * (1) For each additional implemented activation functions in `activation_func.py` you get 1 bonus point (max 4 points). Test your implementation in this notebook and observe effects on your networks. Keep an eye on your layer initialization.
  * (2) Implement `Dropout` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (5) Implement `Batchnorm` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (4) Implement another optimization algorithm in `optimizer.py`. Train one of your models with that new optimizer.
  * (5) Do something extra, up to 5 points.  
  
Please document thoroughly and explain what you do in your experiments, so that work in the notebook is comprehensible, else no points are given.

Fragen
* Sollen wir nur für Pool Layer nur Max forward und backward implementieren? Falls nein, wie solen wir das mit der backward implementierung machen?
* Siehe in layers.py: 
    * In Pooling.backward: What is if not only one pixel has the max value?
    * In Conv Layer: soll zu jedem Pixel bias hinzugefügt werden oder erst nach sum?
    * In Dropout: maske für alle images gleich oder für jedes eine andere? Und wie sollen wir damit umgehen, wenn wir vorhersagen wollen?
    * Braucht man nach Dropout oder Pool Layer eine Aktivierungsfunktion?
    * Pooling Layer gibt immer gleichen loss zurück, woran kann das liegen?

# Requirements

## Python-Modules

In [1]:
%load_ext autoreload
%autoreload 2
# custom 
from htw_nn_framework.networks import NeuralNetwork
from htw_nn_framework.layer import *
from htw_nn_framework.activation_func import *
from htw_nn_framework.loss_func import *
from htw_nn_framework.optimizer import *
from htw_nn_framework.cifar import *
from htw_nn_framework.initializer import *

# third party
from deep_teaching_commons.data.fundamentals.mnist import Mnist
import numpy as np

## Data

In [2]:
# create mnist loader from deep_teaching_commons
mnist_loader = Mnist(data_dir='data')

# load all data, labels are one-hot-encoded, images are flatten and pixel squashed between [0,1]
train_images, train_labels, test_images, test_labels = mnist_loader.get_all_data(flatten=False, one_hot_enc=False, normalized=True)
print(train_images.shape, train_labels.shape)

# reshape to match generell framework architecture 
train_images, test_images = train_images.reshape(60000, 1, 28, 28), test_images.reshape(10000, 1, 28, 28)            
print(train_images.shape, train_labels.shape)

# shuffle training data
shuffle_index = np.random.permutation(60000)
train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]

auto download is active, attempting download
mnist data directory already exists, download aborted
(60000, 28, 28) (60000,)
(60000, 1, 28, 28) (60000,)


# MNIST Fully Connected Network Example
This model and optimization is taken from `framework_exercise.ipynb` as an example for a typical pipeline using the framework files.

In [3]:
train_images_batch, train_labels_batch, test_images_batch, test_labels_batch = train_images[:100,:,:,:], train_labels[:100], test_images[:100,:,:,:], test_labels[:100]

In [4]:
# Design a three hidden layer architecture with dense layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.374715656182273 :: Training = 0.36 :: Test = 0.22
Epoch 2
Loss = 2.395817319196847 :: Training = 0.37 :: Test = 0.28
Epoch 3
Loss = 1.715375971645197 :: Training = 0.6 :: Test = 0.34
Epoch 4
Loss = 1.6932746995957566 :: Training = 0.59 :: Test = 0.4
Epoch 5
Loss = 1.6564738644118353 :: Training = 0.83 :: Test = 0.57
Epoch 6
Loss = 0.9413432878258848 :: Training = 0.88 :: Test = 0.66
Epoch 7
Loss = 0.29728511913975375 :: Training = 0.96 :: Test = 0.71
Epoch 8
Loss = 0.2420351258434986 :: Training = 1.0 :: Test = 0.72
Epoch 9
Loss = 0.057624874393098525 :: Training = 1.0 :: Test = 0.76
Epoch 10
Loss = 0.06200791410649452 :: Training = 1.0 :: Test = 0.76


# Todo: Your Extensions and Experiments
# MNIST
## Convolutional Neural Network

In [5]:
# Design a one hidden layer architecture with conv layer
# and ReLU as activation function
def cnn_mnist():
    hidden_01 = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    flat = Flatten()
    hidden_02 = FullyConnected(784, 1500)
    relu_02 = ReLU()
    ouput = FullyConnected(1500, 10)
    return [hidden_01, relu_01, flat, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
cnn = NeuralNetwork(cnn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn = Optimizer.sgd(cnn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.001, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.2850633905218087 :: Training = 0.15 :: Test = 0.16
Epoch 2
Loss = 2.191314623755756 :: Training = 0.36 :: Test = 0.23
Epoch 3
Loss = 2.090857471811215 :: Training = 0.45 :: Test = 0.22
Epoch 4
Loss = 1.9592645047913169 :: Training = 0.53 :: Test = 0.3
Epoch 5
Loss = 1.7882813482648166 :: Training = 0.61 :: Test = 0.32
Epoch 6
Loss = 1.564081062047945 :: Training = 0.72 :: Test = 0.38
Epoch 7
Loss = 1.2974976819475923 :: Training = 0.81 :: Test = 0.53
Epoch 8
Loss = 1.055454023142705 :: Training = 0.9 :: Test = 0.58
Epoch 9
Loss = 0.8820211329782413 :: Training = 0.93 :: Test = 0.64
Epoch 10
Loss = 0.7107594834843316 :: Training = 0.95 :: Test = 0.68


## Convolutional Neural Network with Pooling

In [6]:
# Design a one hidden layer architecture with conv layer, pooling layer
# and ReLU as activation function
def cnn_pool_mnist():
    hidden_01 = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    pool = Pool(train_images_batch.shape, np.max, 2, 1)
    flat = Flatten()
    ouput = FullyConnected(729, 10)
    return [hidden_01, relu_01, pool, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_pool = NeuralNetwork(cnn_pool_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_pool = Optimizer.sgd(cnn_pool, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.489811424969117 :: Training = 0.51 :: Test = 0.43
Epoch 2
Loss = 1.486813771462919 :: Training = 0.79 :: Test = 0.66
Epoch 3
Loss = 0.8937381651867587 :: Training = 0.6 :: Test = 0.44
Epoch 4
Loss = 1.539132698737083 :: Training = 0.9 :: Test = 0.71
Epoch 5
Loss = 0.2405226618158861 :: Training = 0.97 :: Test = 0.7
Epoch 6
Loss = 0.6782434847612457 :: Training = 0.78 :: Test = 0.63
Epoch 7
Loss = 2.999916865416738 :: Training = 0.74 :: Test = 0.53
Epoch 8
Loss = 0.9693661839482216 :: Training = 0.94 :: Test = 0.75
Epoch 9
Loss = 0.15672145680932356 :: Training = 0.98 :: Test = 0.74
Epoch 10
Loss = 0.07406938797170083 :: Training = 1.0 :: Test = 0.78


## Fully Conected Neural Network with Dropout

In [7]:
# Design a two hidden layer architecture with dense layer, dropout layer
# and ReLU as activation function
def fcn_dropout_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    dropout = Dropout()
    hidden_02 = FullyConnected(500, 100)
    relu_02 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, dropout, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_dropout = NeuralNetwork(fcn_dropout_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_dropout = Optimizer.sgd(fcn_dropout, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.8152648410173406 :: Training = 0.58 :: Test = 0.4
Epoch 2
Loss = 1.653268114374418 :: Training = 0.75 :: Test = 0.54
Epoch 3
Loss = 2.1243440661361386 :: Training = 0.53 :: Test = 0.43
Epoch 4
Loss = 1.9804486064833529 :: Training = 0.57 :: Test = 0.45
Epoch 5
Loss = 1.7400926014109637 :: Training = 0.74 :: Test = 0.6
Epoch 6
Loss = 1.185099790152633 :: Training = 0.88 :: Test = 0.67
Epoch 7
Loss = 1.0147737936988228 :: Training = 0.86 :: Test = 0.62
Epoch 8
Loss = 0.5700326590710398 :: Training = 0.93 :: Test = 0.69
Epoch 9
Loss = 0.6820014922001253 :: Training = 0.95 :: Test = 0.67
Epoch 10
Loss = 0.3162751866790271 :: Training = 1.0 :: Test = 0.68


# CIFAR10

In [8]:
Cifar.load()

Data already downloaded


In [9]:
tr_images, tr_labels, te_images, te_labels = Cifar.get()

/storage/neural-networks/Assignment_03/htw_nn_framework/cifar-10-batches-py/data_batch_1
/storage/neural-networks/Assignment_03/htw_nn_framework/cifar-10-batches-py/data_batch_2
/storage/neural-networks/Assignment_03/htw_nn_framework/cifar-10-batches-py/data_batch_3
/storage/neural-networks/Assignment_03/htw_nn_framework/cifar-10-batches-py/data_batch_4
/storage/neural-networks/Assignment_03/htw_nn_framework/cifar-10-batches-py/data_batch_5


In [10]:
# shuffle training data
shuffle_index = np.random.permutation(len(tr_images))
tr_images, tr_labels = tr_images[shuffle_index], tr_labels[shuffle_index]

tr_images_batch, tr_labels_batch, te_images_batch, te_labels_batch = tr_images[:100,:,:,:], tr_labels[:100], te_images[:100,:,:,:], te_labels[:100]

## Fully Connected Network

In [11]:
# Design a three hidden layer architecture with dense layer
# and ReLU as activation function
def fcn_cifar():
    flat = Flatten()
    hidden_01 = FullyConnected(3072, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_c10 = NeuralNetwork(fcn_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_c10 = Optimizer.sgd(fcn_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=50, learning_rate=0.001, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

Epoch 1
Loss = 3.2050775279514725 :: Training = 0.11 :: Test = 0.1
Epoch 2
Loss = 3.011559622419788 :: Training = 0.14 :: Test = 0.15
Epoch 3
Loss = 2.3160568939061332 :: Training = 0.21 :: Test = 0.11
Epoch 4
Loss = 2.2307650724017205 :: Training = 0.37 :: Test = 0.26
Epoch 5
Loss = 2.125600435479874 :: Training = 0.37 :: Test = 0.26
Epoch 6
Loss = 2.078425141977356 :: Training = 0.49 :: Test = 0.31
Epoch 7
Loss = 1.9929621590876165 :: Training = 0.4 :: Test = 0.18
Epoch 8
Loss = 2.083024423177691 :: Training = 0.41 :: Test = 0.26
Epoch 9
Loss = 2.0006847247076744 :: Training = 0.3 :: Test = 0.21
Epoch 10
Loss = 2.2647431238408116 :: Training = 0.2 :: Test = 0.12
Epoch 11
Loss = 2.0206628191039777 :: Training = 0.38 :: Test = 0.27
Epoch 12
Loss = 1.810956127629157 :: Training = 0.43 :: Test = 0.26
Epoch 13
Loss = 1.8284005897026436 :: Training = 0.46 :: Test = 0.21
Epoch 14
Loss = 1.6347566910556055 :: Training = 0.32 :: Test = 0.2
Epoch 15
Loss = 2.158371192572245 :: Training = 0.26 

## Convolutional Neural Network

In [13]:
# Design a one hidden layer architecture with conv layer
# and ReLU as activation function
def cnn_cifar():
    conv = Conv(tr_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    flat = Flatten()
    ouput = FullyConnected(1024, 10)
    return [conv, relu_01, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_c10 = NeuralNetwork(cnn_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_c10 = Optimizer.sgd(cnn_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

Epoch 1
Loss = 2.302611298239187 :: Training = 0.17 :: Test = 0.16
Epoch 2
Loss = 2.296164235318004 :: Training = 0.17 :: Test = 0.16
Epoch 3
Loss = 2.2912339860886752 :: Training = 0.17 :: Test = 0.16
Epoch 4
Loss = 2.2875029678894325 :: Training = 0.17 :: Test = 0.16
Epoch 5
Loss = 2.284720066592887 :: Training = 0.17 :: Test = 0.16
Epoch 6
Loss = 2.2826847456867956 :: Training = 0.17 :: Test = 0.16
Epoch 7
Loss = 2.2812155066488415 :: Training = 0.17 :: Test = 0.16
Epoch 8
Loss = 2.2801752293490347 :: Training = 0.17 :: Test = 0.16
Epoch 9
Loss = 2.2794518576889704 :: Training = 0.17 :: Test = 0.16
Epoch 10
Loss = 2.279001287718948 :: Training = 0.17 :: Test = 0.16


## Convolutional Neural Network with Pooling

In [16]:
def cnn_pool_cifar():
    conv = Conv(tr_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    pool = Pool(tr_images_batch.shape, np.max, 2, 1)
    flat = Flatten()
    ouput = FullyConnected(961, 10)
    return [conv, relu_01, pool, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_pool_c10 = NeuralNetwork(cnn_pool_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_pool_c10 = Optimizer.sgd(cnn_pool_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=30, learning_rate=0.001, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

Epoch 1
Loss = 2.3345305906717284 :: Training = 0.16 :: Test = 0.12
Epoch 2
Loss = 2.32878022929078 :: Training = 0.19 :: Test = 0.12
Epoch 3
Loss = 2.324507580853724 :: Training = 0.2 :: Test = 0.13
Epoch 4
Loss = 2.3169866420230623 :: Training = 0.2 :: Test = 0.16
Epoch 5
Loss = 2.305752171928516 :: Training = 0.2 :: Test = 0.16
Epoch 6
Loss = 2.289041711114825 :: Training = 0.21 :: Test = 0.15
Epoch 7
Loss = 2.2732884881915734 :: Training = 0.25 :: Test = 0.15
Epoch 8
Loss = 2.244754289155251 :: Training = 0.3 :: Test = 0.15
Epoch 9
Loss = 2.2226558763260735 :: Training = 0.31 :: Test = 0.16
Epoch 10
Loss = 2.202152812345323 :: Training = 0.29 :: Test = 0.16
Epoch 11
Loss = 2.183739159350683 :: Training = 0.3 :: Test = 0.15
Epoch 12
Loss = 2.167967877684971 :: Training = 0.31 :: Test = 0.15
Epoch 13
Loss = 2.153339349026197 :: Training = 0.33 :: Test = 0.13
Epoch 14
Loss = 2.1401278292213695 :: Training = 0.35 :: Test = 0.14
Epoch 15
Loss = 2.1280550011406603 :: Training = 0.36 :: T

## Fully Conected Neural Network with Dropout

In [17]:
# Design a two hidden layer architecture with dense layer, dropout layer
# and ReLU as activation function
def fcn_dropout_cifar():
    flat = Flatten()
    hidden_01 = FullyConnected(3072, 500)
    relu_01 = ReLU()
    dropout = Dropout()
    hidden_02 = FullyConnected(500, 100)
    relu_02 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, dropout, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_dropout_c10 = NeuralNetwork(fcn_dropout_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_dropout_c10 = Optimizer.sgd(fcn_dropout_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=40, learning_rate=0.001, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

Epoch 1
Loss = 3.171042149551644 :: Training = 0.11 :: Test = 0.15
Epoch 2
Loss = 3.3207889598836373 :: Training = 0.14 :: Test = 0.1
Epoch 3
Loss = 3.124752057661935 :: Training = 0.14 :: Test = 0.15
Epoch 4
Loss = 2.4397542517592354 :: Training = 0.18 :: Test = 0.16
Epoch 5
Loss = 2.321990831515732 :: Training = 0.31 :: Test = 0.19
Epoch 6
Loss = 2.3172580996097265 :: Training = 0.33 :: Test = 0.18
Epoch 7
Loss = 2.2882203194427713 :: Training = 0.26 :: Test = 0.23
Epoch 8
Loss = 2.1499055493228174 :: Training = 0.33 :: Test = 0.2
Epoch 9
Loss = 2.3541742678051456 :: Training = 0.32 :: Test = 0.23
Epoch 10
Loss = 2.2537620045545275 :: Training = 0.41 :: Test = 0.2
Epoch 11
Loss = 2.1118546376174376 :: Training = 0.4 :: Test = 0.3
Epoch 12
Loss = 2.169984427505885 :: Training = 0.44 :: Test = 0.24
Epoch 13
Loss = 2.20394506661958 :: Training = 0.44 :: Test = 0.26
Epoch 14
Loss = 2.295369757625976 :: Training = 0.33 :: Test = 0.16
Epoch 15
Loss = 2.174530732548236 :: Training = 0.51 ::